In [1]:
import pandas as pd 
import numpy as np
from constants import headers_list, migrant_keywords

In [2]:
#Starting by writing in the CSV files as dataframes

def csv2df(filename):
    df = pd.read_csv(filename, sep='\t')
    return df

In [3]:
#Adding headers to the data frames

def add_head(df, headers):
    new_df = df.copy()
    new_df.columns = headers
    return new_df 

In [4]:
#Drop duplicates from CSV

In [5]:
def nodupes(df):
    new_df = df.copy().drop_duplicates(subset=['SOURCEURL'])
    return new_df

In [6]:
def get_us_df(df):
    us_df = df.copy().loc[(df["ActionGeo_CountryCode"].str.contains("US"))&
    pd.notnull(df['ActionGeo_CountryCode']),
    ['ActionGeo_FullName', "MonthYear", "Year", "AvgTone", "ActionGeo_CountryCode", "SOURCEURL"]
    ]
    return us_df

In [7]:
import re
def regex_search(keywords, text):
    pattern = re.compile(f'.*({"|".join(map(re.escape, keywords))}).*', flags=re.IGNORECASE)
    return bool(pattern.match(text))

In [11]:
def process_data(filename):
    df = csv2df(filename)
    df = add_head(df, headers_list)
    df = nodupes(df)
    df = df[df['SOURCEURL'].apply(lambda x: regex_search(migrant_keywords, x))]
    us_df = get_us_df(df)
    non_us_df = df[df["ActionGeo_CountryCode"] != "US"]
    return us_df["AvgTone"].tolist(), non_us_df["AvgTone"].tolist()
    

In [25]:
def filter_year(year):
    try:
        year = int(year)
    except ValueError: #not a year 
        return False
    if year == 2016:
        return True 
    elif year == 2020:
        return True
    return False

In [13]:
# my_filename = "20231129.export.CSV"
# process_data(my_filename)

In [ ]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import pandas as pd
from tqdm import tqdm

def download_and_process_files(url, link_root):
    # Get the list of zip files
    all_us_tones = []
    all_non_us_tones = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = [link.get('href') for link in soup.find_all('a') if link.get('href').endswith('.zip')][1:]
    links = [l for l in links if filter_year(l[:4])]

    for link in tqdm(links):
#         # Download zip file
        response = requests.get(link_root + link)
        zip_filename = link.split('/')[-1]
        with open(zip_filename, 'wb') as f:
            f.write(response.content)

#         # Unzip file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(".")

#         # Process data (placeholder for your analysis)
#         # Example: read CSV file and do analysis
        csv_filename = zip_filename.replace('.zip', '')
        us_tones, non_us_tones = process_data(csv_filename)
        all_us_tones += us_tones
        all_non_us_tones += non_us_tones
        # Data analysis code goes here

#         # Delete files
        os.remove(zip_filename)
        os.remove(csv_filename)
    return all_us_tones, all_non_us_tones

# URL of the GDELT Project's events page
url = "http://data.gdeltproject.org/events/index.html"
baseurl = "http://data.gdeltproject.org/events/"
us_tones, non_us_tones = download_and_process_files(url, baseurl)

  0%|          | 2/732 [00:02<15:36,  1.28s/it]/var/folders/lq/3m9k21yj1j1_qk_j3f0lkwhh0000gn/T/ipykernel_13332/730791947.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep='\t')
  1%|          | 7/732 [00:07<11:37,  1.04it/s]/var/folders/lq/3m9k21yj1j1_qk_j3f0lkwhh0000gn/T/ipykernel_13332/730791947.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep='\t')
  1%|          | 9/732 [00:10<14:23,  1.19s/it]/var/folders/lq/3m9k21yj1j1_qk_j3f0lkwhh0000gn/T/ipykernel_13332/730791947.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep='\t')
  1%|▏         | 10/732 [00:11<15:19,  1.27s/it]/var/folders/lq/3m9k21yj1j1_qk_j3f0lkwhh0000gn/T/ipykernel_13332/730791947.py:4: DtypeWarning: Columns (8,11,14) have mixed types. Specify dtype opt

In [20]:
len(non_us_tones), len(us_tones)

(1543, 582)